In [ ]:
# Cell pertama - Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime

print("Libraries berhasil di-import!") 

In [ ]:
# Setup untuk data analysis
plt.style.use('seaborn-v0_8')  # Bikin plot lebih cantik
pd.set_option('display.max_columns', None)  # Tampilin semua kolom

# Cek working directory
import os
print("Current directory:", os.getcwd())
print("Files in directory:", os.listdir('.'))

In [ ]:
# Load Raw data
try:
    mrt_data = pd.read_csv('Data/Raw/Jumlah Penumpang Mass Rapid Transit (MRT) Jakarta, 2024 (1).csv')
    lrt_data = pd.read_csv('Data/Raw/Jumlah Penumpang Light Rail Transit (LRT) Jakarta, 2024.csv')  
    transjakarta_data = pd.read_csv('Data/Raw/Jumlah Penumpang Bus Transjakarta Menurut Bulan, 2024.csv')
    
    print("✅ Semua data berhasil di-load!")
    print(f"MRT data shape: {mrt_data.shape}")
    print(f"LRT data shape: {lrt_data.shape}")
    print(f"Transjakarta data shape: {transjakarta_data.shape}")
    
    print("\n📊 Preview MRT data:")
    print(mrt_data.head())

except Exception as e:
    print("❌ Error:", e)


In [ ]:
# Cell baru - Data Cleaning yang BENAR
print("🔧 CLEANING DATA TRANSPORTASI JAKARTA - FIXED")
print("="*50)

def clean_transport_data_fixed(df, transport_name):
    """Function untuk clean data transportasi - FIXED VERSION"""
    print(f"\n📊 Cleaning {transport_name} data...")
    
    # Berdasarkan debug:
    # Row 2 (index 2) = nama bulan (kolom 1 ke atas)
    # Row 3 (index 3) = data penumpang (kolom 1 ke atas)
    
    # Ambil data bulan (row 2, kolom 1 ke atas)
    months = df.iloc[2, 1:].values  # Januari, Februari, dst
    
    # Ambil data penumpang (row 3, kolom 1 ke atas)  
    passengers = df.iloc[3, 1:].values  # 3133700, 2595293, dst
    
    # Bikin DataFrame baru yang rapi
    clean_df = pd.DataFrame({
        'Bulan': months,
        'Penumpang': passengers,
        'Moda': transport_name
    })
    
    # Clean data yang kosong (- jadi NaN)
    clean_df['Penumpang'] = clean_df['Penumpang'].replace('-', np.nan)
    
    # Convert ke numeric
    clean_df['Penumpang'] = pd.to_numeric(clean_df['Penumpang'], errors='coerce')
    
    # Hapus data yang kosong dan 'Tahunan'
    clean_df = clean_df.dropna(subset=['Penumpang'])
    clean_df = clean_df[clean_df['Bulan'] != 'Tahunan']
    
    print(f"✅ {transport_name} berhasil di-clean!")
    print(f"Data points: {len(clean_df)}")
    print(clean_df.head())
    
    return clean_df

# Clean semua data dengan function yang fixed
mrt_clean = clean_transport_data_fixed(mrt_data, 'MRT')
lrt_clean = clean_transport_data_fixed(lrt_data, 'LRT') 
transjakarta_clean = clean_transport_data_fixed(transjakarta_data, 'Transjakarta')

print("\n🎯 SUMMARY CLEAN DATA:")
print("="*50)
print(f"MRT: {len(mrt_clean)} bulan data")
print(f"LRT: {len(lrt_clean)} bulan data") 
print(f"Transjakarta: {len(transjakarta_clean)} bulan data")

# Gabung semua data jadi satu DataFrame
all_data = pd.concat([mrt_clean, lrt_clean, transjakarta_clean], ignore_index=True)
print(f"\nTotal combined data: {len(all_data)} rows")
print("\n📈 Sample gabungan data:")
print(all_data.head(10))

In [ ]:
# Cell baru - Analisis Dasar dan Pie Chart
import os

print("ANALISIS DATA TRANSPORTASI JAKARTA 2024")
print("="*50)

# 1. SUMMARY STATISTICS
print("1. RINGKASAN DATA (Jan-Sep 2024):")
summary = all_data.groupby('Moda')['Penumpang'].agg(['sum', 'mean', 'min', 'max'])
summary.columns = ['Total Penumpang', 'Rata-rata/Bulan', 'Minimum', 'Maksimum']
summary['Total Penumpang'] = summary['Total Penumpang'].map('{:,.0f}'.format)
summary['Rata-rata/Bulan'] = summary['Rata-rata/Bulan'].map('{:,.0f}'.format)
print(summary)

# 2. PERSENTASE KONTRIBUSI (untuk Pie Chart)
print("\n2. KONTRIBUSI PENUMPANG PER MODA:")
total_penumpang = all_data.groupby('Moda')['Penumpang'].sum()
persentase = (total_penumpang / total_penumpang.sum() * 100).round(2)
for moda, persen in persentase.items():
    print(f"{moda}: {persen}% ({total_penumpang[moda]:,.0f} penumpang)")

# Buat folder images/ kalau belum ada
os.makedirs("images", exist_ok=True)

# 3. PIE CHART
plt.figure(figsize=(10, 8))
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
plt.pie(total_penumpang.values, labels=total_penumpang.index, autopct='%1.1f%%', 
        startangle=90, colors=colors, explode=(0.05, 0.05, 0.05))
plt.title('Distribusi Penumpang Transportasi Publik Jakarta\n(Januari - September 2024)', 
          fontsize=14, fontweight='bold', pad=20)
plt.axis('equal')
plt.tight_layout()

# Simpan chart ke file PNG (di folder images)
plt.savefig("images/grafik_transportasi.png", dpi=300)

# Tetap tampilkan di notebook
plt.show()

# 4. PIVOT TABLE untuk analisis selanjutnya
pivot_data = all_data.pivot(index='Bulan', columns='Moda', values='Penumpang')
month_order = ['Januari', 'Februari', 'Maret', 'April', 'Mei', 'Juni', 'Juli', 'Agustus', 'September']
pivot_data = pivot_data.reindex(month_order)

print("\n3. DATA BULANAN (untuk analisis growth):")
print(pivot_data)


In [ ]:
# Cell baru - Month-over-Month Growth Analysis
import os

print("MONTH-OVER-MONTH GROWTH ANALYSIS")
print("="*50)

# 1. Hitung MoM Growth Rate
def calculate_mom_growth(df):
    """Hitung month-over-month growth rate"""
    growth_rates = {}
    
    for moda in df.columns:
        values = df[moda].dropna()
        growth = []
        
        for i in range(1, len(values)):
            prev_month = values.iloc[i-1] 
            current_month = values.iloc[i]
            mom_growth = ((current_month - prev_month) / prev_month) * 100
            growth.append(mom_growth)
        
        growth_rates[moda] = growth
    
    return growth_rates

# Hitung growth rates
growth_rates = calculate_mom_growth(pivot_data)

# 2. Buat DataFrame untuk growth rates
growth_months = month_order[1:]  # Skip Januari (no previous month)
growth_df = pd.DataFrame(growth_rates, index=growth_months)

print("1. MONTH-OVER-MONTH GROWTH RATES (%):")
print(growth_df.round(2))

# Pastikan folder images/ ada
os.makedirs("images", exist_ok=True)

# 3. Visualisasi MoM Growth
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Plot 1: Line chart penumpang bulanan
pivot_data.plot(kind='line', ax=ax1, marker='o', linewidth=2)
ax1.set_title('Tren Penumpang Bulanan 2024', fontsize=14, fontweight='bold')
ax1.set_ylabel('Jumlah Penumpang')
ax1.legend(title='Moda Transportasi')
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# Plot 2: Bar chart growth rates
growth_df.plot(kind='bar', ax=ax2, width=0.8)
ax2.set_title('Month-over-Month Growth Rate (%)', fontsize=14, fontweight='bold')
ax2.set_ylabel('Growth Rate (%)')
ax2.legend(title='Moda Transportasi')
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.grid(True, alpha=0.3)
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()

# Simpan grafik ke file
plt.savefig("images/mom_growth_analysis.png", dpi=300)

# Tetap tampilkan di notebook
plt.show()

# 4. Growth insights
print("\n2. GROWTH INSIGHTS:")
for moda in growth_df.columns:
    avg_growth = growth_df[moda].mean()
    max_growth = growth_df[moda].max()
    min_growth = growth_df[moda].min()
    
    max_month = growth_df[growth_df[moda] == max_growth].index[0]
    min_month = growth_df[growth_df[moda] == min_growth].index[0]
    
    print(f"\n{moda}:")
    print(f"  - Rata-rata growth: {avg_growth:.2f}%")
    print(f"  - Growth tertinggi: {max_growth:.2f}% ({max_month})")
    print(f"  - Growth terendah: {min_growth:.2f}% ({min_month})")


In [ ]:
# Cell baru - Forecasting untuk 3-6 bulan ke depan (dengan save gambar)
print("FORECASTING PENUMPANG 3-6 BULAN KE DEPAN")
print("="*50)

from sklearn.linear_model import LinearRegression
import numpy as np

# 1. Setup data untuk forecasting
def prepare_forecast_data(df):
    """Prepare data untuk forecasting"""
    forecast_data = {}
    
    for moda in df.columns:
        # Get clean data (drop NaN)
        clean_data = df[moda].dropna()
        
        # Create time series (1, 2, 3, ..., 9 untuk Jan-Sep)
        X = np.array(range(1, len(clean_data) + 1)).reshape(-1, 1)
        y = clean_data.values
        
        forecast_data[moda] = {'X': X, 'y': y, 'months': clean_data.index.tolist()}
    
    return forecast_data

forecast_data = prepare_forecast_data(pivot_data)

# 2. Training model dan prediksi
predictions = {}

print("1. MODEL TRAINING & PREDICTION:")
for moda, data in forecast_data.items():
    # Train linear regression model
    model = LinearRegression()
    model.fit(data['X'], data['y'])
    
    # Predict 6 bulan ke depan (Oct 2024 - Mar 2025)
    future_months = np.array(range(10, 16)).reshape(-1, 1)  # 10=Oct, 11=Nov, ..., 15=Mar
    future_pred = model.predict(future_months)
    
    # Calculate R-squared
    r2 = model.score(data['X'], data['y'])
    
    predictions[moda] = {
        'model': model,
        'predictions': future_pred,
        'r2': r2
    }
    
    print(f"\n{moda}:")
    print(f"  - R-squared: {r2:.3f}")
    print(f"  - Trend: {'Naik' if model.coef_[0] > 0 else 'Turun'} ({model.coef_[0]:.0f} penumpang/bulan)")

# 3. Buat forecast table
future_months_names = ['Oktober', 'November', 'Desember', 'Januari 2025', 'Februari 2025', 'Maret 2025']

forecast_df = pd.DataFrame()
for moda, pred in predictions.items():
    forecast_df[moda] = pred['predictions']
forecast_df.index = future_months_names

print("\n2. PREDIKSI PENUMPANG (6 BULAN KE DEPAN):")
forecast_display = forecast_df.copy()
for col in forecast_display.columns:
    forecast_display[col] = forecast_display[col].apply(lambda x: f"{x:,.0f}")
print(forecast_display)

# 4. Visualisasi Historical + Forecast
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

all_months = month_order + future_months_names
historical_length = len(month_order)

for i, moda in enumerate(['MRT', 'LRT', 'Transjakarta']):
    ax = axes[i]
    
    # Historical data
    historical_data = pivot_data[moda].values
    ax.plot(range(1, historical_length + 1), historical_data, 
            'o-', color=colors[i], linewidth=2, label='Historical', markersize=6)
    
    # Forecast data
    forecast_data_viz = predictions[moda]['predictions']
    ax.plot(range(historical_length + 1, historical_length + 7), forecast_data_viz, 
            's--', color=colors[i], alpha=0.7, linewidth=2, label='Forecast', markersize=6)
    
    # Styling
    ax.set_title(f'{moda} Forecast', fontweight='bold')
    ax.set_ylabel('Penumpang')
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    # X-axis labels
    tick_positions = list(range(1, len(all_months) + 1))
    ax.set_xticks([1, 3, 5, 7, 9, 11, 13, 15])
    ax.set_xticklabels(['Jan', 'Mar', 'Mei', 'Jul', 'Sep', 'Nov', 'Jan 25', 'Mar 25'], rotation=45)

plt.tight_layout()

# SAVE GAMBAR SEBELUM SHOW
plt.savefig('images/forecast_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n3. FORECAST SUMMARY:")
for moda in predictions.keys():
    current_avg = pivot_data[moda].mean()
    forecast_avg = predictions[moda]['predictions'].mean()
    change = ((forecast_avg - current_avg) / current_avg) * 100
    
    print(f"{moda}:")
    print(f"  - Rata-rata Jan-Sep 2024: {current_avg:,.0f}")
    print(f"  - Prediksi rata-rata 6 bulan: {forecast_avg:,.0f}")
    print(f"  - Perubahan: {change:+.1f}%")

print(f"\n✅ Forecast chart saved as 'forecast_analysis.png'")

In [ ]:
# Cell baru - Comprehensive Dashboard dengan Visualisasi Lengkap
print("COMPREHENSIVE DASHBOARD - TRANSPORTASI JAKARTA 2024")
print("="*60)

import matplotlib.patches as mpatches
from matplotlib.gridspec import GridSpec
import seaborn as sns

# Create folder images if not exists
import os
if not os.path.exists('images'):
    os.makedirs('images')

# Setup comprehensive dashboard
fig = plt.figure(figsize=(20, 14))
fig.patch.set_facecolor('white')

# Create grid layout
gs = GridSpec(4, 4, figure=fig, hspace=0.4, wspace=0.3)

# Color palette
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
bg_color = '#f8f9fa'

# Calculate key metrics
total_all = all_data['Penumpang'].sum()
market_share = all_data.groupby('Moda')['Penumpang'].sum().sort_values(ascending=False)
leader = market_share.index[0]
leader_share = (market_share.iloc[0] / total_all * 100)
avg_growth = growth_df.mean().sort_values(ascending=False)
growth_leader = avg_growth.index[0]
monthly_totals = pivot_data.sum(axis=1)
peak_month = monthly_totals.idxmax()

# 1. HEADER - Key Metrics Cards (Top Row)
ax_header = fig.add_subplot(gs[0, :])
ax_header.axis('off')

# Create metric cards
cards_data = [
    ("Total Penumpang", f"{total_all:,.0f}", "Jan-Sep 2024"),
    ("Market Leader", f"{leader}", f"{leader_share:.1f}% share"),
    ("Peak Month", f"{peak_month}", f"{monthly_totals.max():,.0f} total"),
    ("Growth Leader", f"{growth_leader}", f"{avg_growth.iloc[0]:+.1f}%/month")
]

for i, (title, value, subtitle) in enumerate(cards_data):
    x_pos = i * 0.25 + 0.02
    # Card background
    card_bg = mpatches.FancyBboxPatch((x_pos, 0.3), 0.22, 0.4, 
                                     boxstyle="round,pad=0.02",
                                     facecolor=colors[i%3], alpha=0.1,
                                     edgecolor=colors[i%3], linewidth=2)
    ax_header.add_patch(card_bg)
    
    # Card text
    ax_header.text(x_pos + 0.11, 0.6, title, ha='center', va='center', 
                   fontsize=12, fontweight='bold', color='black')
    ax_header.text(x_pos + 0.11, 0.45, value, ha='center', va='center',
                   fontsize=16, fontweight='bold', color=colors[i%3])
    ax_header.text(x_pos + 0.11, 0.35, subtitle, ha='center', va='center',
                   fontsize=10, color='gray')

ax_header.set_xlim(0, 1)
ax_header.set_ylim(0, 1)

# 2. PIE CHART - Market Share (Row 2, Col 1)
ax1 = fig.add_subplot(gs[1, 0])
wedges, texts, autotexts = ax1.pie(market_share.values, labels=market_share.index, 
                                  autopct='%1.1f%%', startangle=90, colors=colors,
                                  explode=(0.05, 0.05, 0.05))
ax1.set_title('Market Share Distribution\n(Jan-Sep 2024)', fontweight='bold', pad=20)
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

# 3. MONTHLY TREND LINE (Row 2, Col 2-4)
ax2 = fig.add_subplot(gs[1, 1:])
for i, moda in enumerate(pivot_data.columns):
    ax2.plot(pivot_data.index, pivot_data[moda], 'o-', linewidth=3, 
             markersize=8, color=colors[i], label=moda)
ax2.set_title('Monthly Ridership Trend 2024', fontweight='bold', fontsize=14)
ax2.set_ylabel('Penumpang (juta)', fontweight='bold')
ax2.legend(loc='upper left', frameon=True)
ax2.grid(True, alpha=0.3)
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1e6:.1f}M'))
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)

# Highlight peak month
peak_idx = list(pivot_data.index).index(peak_month)
ax2.axvline(x=peak_idx, color='red', linestyle='--', alpha=0.7, linewidth=2)
ax2.text(peak_idx, ax2.get_ylim()[1]*0.9, f'Peak: {peak_month}', 
         ha='center', color='red', fontweight='bold')

# 4. GROWTH HEATMAP (Row 3, Col 1-2)
ax3 = fig.add_subplot(gs[2, :2])
growth_heatmap = sns.heatmap(growth_df.T, annot=True, fmt='.1f', cmap='RdYlGn', 
                            center=0, ax=ax3, cbar_kws={'label': 'Growth Rate (%)'})
ax3.set_title('Month-over-Month Growth Rate (%)', fontweight='bold')
ax3.set_ylabel('Transportation Mode', fontweight='bold')
ax3.set_xlabel('Month', fontweight='bold')

# 5. FORECAST COMPARISON (Row 3, Col 3-4)
ax4 = fig.add_subplot(gs[2, 2:])
current_avg = pivot_data.mean()
forecast_avg = pd.Series({moda: predictions[moda]['predictions'].mean() 
                         for moda in predictions.keys()})
x_pos = range(len(current_avg))
width = 0.35

bars1 = ax4.bar([p - width/2 for p in x_pos], current_avg.values/1e6, width, 
               label='Current Avg (2024)', color=colors, alpha=0.7)
bars2 = ax4.bar([p + width/2 for p in x_pos], forecast_avg.values/1e6, width,
               label='Forecast Avg (Next 6M)', color=colors, alpha=1.0,
               edgecolor='black', linewidth=1)

ax4.set_title('Current vs Forecast Average', fontweight='bold')
ax4.set_ylabel('Penumpang (juta)', fontweight='bold')
ax4.set_xticks(x_pos)
ax4.set_xticklabels(current_avg.index)
ax4.legend()
ax4.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + 0.05,
             f'{height:.1f}M', ha='center', va='bottom', fontsize=10)
for bar in bars2:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + 0.05,
             f'{height:.1f}M', ha='center', va='bottom', fontsize=10, fontweight='bold')

# 6. INSIGHTS & RECOMMENDATIONS (Bottom Row)
ax5 = fig.add_subplot(gs[3, :])
ax5.axis('off')

# Create insights text box
insights_text = f"""
KEY INSIGHTS & RECOMMENDATIONS

📊 MARKET ANALYSIS
• {leader} dominates the market with {leader_share:.1f}% ridership share ({market_share.iloc[0]:,.0f} passengers)
• Peak ridership occurred in {peak_month} with {monthly_totals.max():,.0f} total passengers across all modes
• {growth_leader} shows the strongest growth momentum at {avg_growth.iloc[0]:+.1f}% monthly average

📈 GROWTH PATTERNS  
• Overall positive growth trend across all transportation modes
• Average monthly growth rate: {avg_growth.mean():+.1f}% per mode
• Seasonal peak suggests increased mobility during {peak_month}

🎯 STRATEGIC RECOMMENDATIONS
• Infrastructure Development: Focus expansion efforts on {growth_leader} given its strong growth trajectory
• Capacity Management: Evaluate and increase capacity during peak months (especially {peak_month})
• Service Integration: Implement unified payment system across all three transportation modes
• Marketing Strategy: Develop targeted campaigns for underutilized modes to balance ridership distribution

📋 FORECAST OUTLOOK (Next 6 Months)
• Predicted continued growth across all modes with average {np.mean([(predictions[moda]['predictions'].mean() - pivot_data[moda].mean()) / pivot_data[moda].mean() * 100 for moda in predictions.keys()]):+.1f}% increase
• Model accuracy: R² = {np.mean([p['r2'] for p in predictions.values()]):.3f} (indicating reliable forecast confidence)
• Expected total ridership: {sum(predictions[moda]['predictions'].sum() for moda in predictions.keys()):,.0f} passengers over 6 months
"""

# Create text box with background
text_box = ax5.text(0.02, 0.98, insights_text, transform=ax5.transAxes, 
                   fontsize=11, verticalalignment='top', fontfamily='monospace',
                   bbox=dict(boxstyle='round,pad=1', facecolor='#f8f9fa', 
                            alpha=0.9, edgecolor='gray', linewidth=1))

ax5.set_xlim(0, 1)
ax5.set_ylim(0, 1)

# Main title
fig.suptitle('JAKARTA PUBLIC TRANSPORTATION ANALYTICS DASHBOARD 2024\nComprehensive Analysis & Strategic Insights', 
             fontsize=18, fontweight='bold', y=0.97)

# Add footer
fig.text(0.5, 0.02, 'Data Source: Jakarta Transportation Authority | Analysis Period: January - September 2024 | Forecast Period: October 2024 - March 2025',
         ha='center', fontsize=10, style='italic', color='gray')

plt.tight_layout()

# Save the comprehensive dashboard
plt.savefig('images/comprehensive_dashboard_summary.png', dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none')
plt.show()

print(f"\n✅ COMPREHENSIVE DASHBOARD COMPLETED!")
print(f"📁 Saved as: 'images/comprehensive_dashboard_summary.png'")
print(f"📊 Dashboard includes: Market share, trends, growth analysis, forecasts & insights")
print(f"📈 Total visualizations: 5 charts + key metrics cards + strategic recommendations")